In [1]:
import pandas as pd
import numpy as np
data = {
    'Patient_ID': range(1,11),
    'Age': [30,45,50,35,60,55,40,25,65,45],
    'Blood_Pressure': ['High','Low','High','Low','High','Low','High','Low','High','Low'],
    'Cholesterol': ['High','Normal','High','Normal','High','Normal','High','Normal','High','Normal'],
    'Diagnosis': ['Sick','Healthy','Sick','Healthy','Sick','Healthy','Sick','Healthy','Sick','Healthy']
}
df = pd.DataFrame(data)

In [2]:
class Node:
    def __init__(self, feature = None, threshold=None, left = None, right = None, value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def calculate_entropy(y):
    classes = np.unique(y)
    entropy = 0
    for c in classes:
        p= len(y[y==c])/len(y)
        entropy -= p*np.log2(p) if p>0 else 0
    return entropy

def information_gain(y,X_feature):
    parent_entropy = calculate_entropy(y)
    # Calculate the weighted entropy of the children
    values = np.unique(X_feature)
    weighted_entropy = 0
    for value in values:
        mask = X_feature == value
        child_entropy = calculate_entropy(y[mask])
        weight = len(y[mask])/len(y)
        weighted_entropy+=weight*child_entropy
    return parent_entropy- weighted_entropy



In [7]:
def build_tree(X,y):
    if len(np.unique(y))==1:
        # return Node(value = y[0])
        return Node(value=y.iloc[0])
    gains = [(col,information_gain(y,X[col]))for col in X.columns]
    best_feature = max(gains,key = lambda x:x[1])[0]
    node = Node(feature = best_feature)

    for value in [0,1]:
        mask  = X[best_feature]==value
        if len(X[mask])>0:
            child = build_tree(X[mask].drop(best_feature,axis=1),y[mask])
            if value == 0:
                node.left = child
            else:
                node.right = child
    return node
def predict(node,x):
    if node.value is not None:
        return node.value
    return predict(node.left,x) if x[node.feature]==0 else predict(node.right,x)

In [8]:
# Example usage
data = {
    'Blood_Pressure': [1,0,1,0,1,0,1,0,1,0],  # High=1, Low=0
    'Cholesterol': [1,0,1,0,1,0,1,0,1,0],     # High=1, Normal=0
    'Diagnosis': [1,0,1,0,1,0,1,0,1,0]        # Sick=1, Healthy=0
}
df = pd.DataFrame(data)

X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']

tree = build_tree(X, y)
test_case = pd.Series({'Blood_Pressure': 1, 'Cholesterol': 1})
prediction = predict(tree, test_case)
print(f"Prediction: {'Sick' if prediction == 1 else 'Healthy'}")

Prediction: Sick
